## Set up

#### Install packages:

In [32]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

#### Set CSVs:

In [15]:
gp_csv = "/Users/securedatascience4/Desktop/GPPractices.csv"
indicators_csv = "/Users/securedatascience4/Desktop/TransparencyIndicatorsGPPerformance.csv"
hours_csv = "/Users/securedatascience4/Desktop/GPOpeningTimes.csv"

#### Read CSVs:

In [16]:
gp = pd.read_csv(gp_csv,
                 header=0,
                 sep="¬",
                 engine="python",
                 encoding='latin-1')
indicators = pd.read_csv(indicators_csv,
                        header=0,
                        sep="¬",
                        engine="python",
                        encoding='latin-1')
hours = pd.read_csv(hours_csv,
                        header=0,
                        sep="¬",
                        engine="python",
                        encoding='latin-1')

#### Drop indicators duplicates:

In [17]:
indicators = indicators.drop_duplicates().reset_index(drop=True)

#### Clean hours dataframe:

In [18]:
hours.rename(columns={'OrganisationId': 'OrganisationID'}, inplace=True)
hours['Open'], hours['Close'] = hours['Times'].str.split('-', 1).str
hours.drop(columns='Times')
hours = hours[['OrganisationID','WeekDay','Open','Close','OpeningTimeType','AdditionalOpeningDate']]
hours['Open'] = pd.to_datetime(hours['Open'], format='%H:%M').dt.time 
hours['Close'] = pd.to_datetime(hours['Close'], format='%H:%M').dt.time

#### Create list of headers for each dataframe:

In [19]:
gp_list = list(gp.columns.values)
indicators_list = list(indicators.columns.values)
hours_list = list(hours.columns.values)

#### Define correct postcodes:

In [20]:
correct_postcodes = gp.City.str.contains("^[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][ABD-HJLNP-UW-Z]{2}$")

## Exploratory analysis of metrics

In [21]:
count_metric = indicators.groupby('MetricName')['OrganisationID'].count().sort_values(ascending=False).reset_index(name='count')

In [22]:
count_metric

,MetricName,count
0,Flu vaccinations - Aged 65 and over group,15810
1,Flu vaccinations - Under 65 At Risk Groups,15808
2,Care Quality Commission Inspection Ratings,15772
3,Patient Satisfaction with GP Practice Opening ...,14225
4,Patient Experience - Confidence and Trust in GP,14225
5,Antibiotic Prescribing,14166
6,Cervical Cancer Screening,13973
7,Percentage of children aged 1 completed primar...,13925
8,"Proportion of children aged 2 with Measles, Mu...",13923
9,Proportion of children aged 2 with Haemophilu...,13923


In [23]:
indicators.groupby('MetricName')['Value']

In [25]:
indicators_pivot = indicators.pivot(index='OrganisationID',columns='MetricName',values='Value')

In [26]:
indicators_pivot.head()

MetricName,Antibiotic Prescribing,Cancer Detection Rate,Care Quality Commission Inspection Ratings,Cervical Cancer Screening,Dementia - Face to Face Reviews,Diabetes - Managing Blood Glucose Level (HbA1c),Flu vaccinations - Aged 65 and over group,Flu vaccinations - Under 65 At Risk Groups,High Blood Pressure Management,Mental Health  Comprehensive Care Planning,Patient Experience - Confidence and Trust in GP,Patient Satisfaction with GP Practice Opening Times,Percentage of children aged 1 completed primary course of 5:1 vaccine,Proportion of children aged 2 with Haemophilus influenzae type b and Meningitis C booster vaccine,"Proportion of children aged 2 with Measles, Mumps and Rubella vaccine",Proportion of children aged 2 with pneumococcal conjugate booster vaccine,Stroke prevention: medication for patients with atrial fibrillation
OrganisationID,,,,,,,,,,,,,,,,,
2915,0.54,60.00%,NaN,Below England expectation,None,Below England value,Below England average,Above England average,Above England value,Below England value,Below England average,Similar to England average,Above England standard,Below England standard,Below England standard,Below England standard,Above England value
3028,1.01,53.30%,NaN,Below England expectation,Below England value,Above England value,Below England average,Below England average,Above England value,Above England value,Below England average,Similar to England average,Below England standard,Below England standard,Below England standard,Below England standard,Above England value
3029,1.15,58.10%,NaN,Below England expectation,Above England value,Above England value,Below England average,Above England average,Above England value,Above England value,Similar to England average,Similar to England average,Below England standard,Below England standard,Below England standard,Below England standard,Above England value
3031,1.01,69.60%,NaN,Below England expectation,Above England value,Below England value,Below England average,Below England average,Above England value,Above England value,Similar to England average,Similar to England average,Below England standard,Below England standard,Below England standard,Below England standard,Above England value
3033,1.00,46.30%,NaN,Below England expectation,Above England value,Above England value,Above England average,Below England average,Above England value,Above England value,Similar to England average,Similar to England average,Below England standard,Above England standard,Above England standard,Above England standard,Above England value


In [27]:
indicators[indicators['MetricName']=='Cancer Detection Rate'].sample(5)

,OrganisationID,OrganisationCode,OrganisationName,MetricName,Value,Text
232621,54671,M85178,Enki Medical Practice,Cancer Detection Rate,33.30%,"18 detected by practice, out of new diagnoses."
332,3052,A81022,Hillside Practice,Cancer Detection Rate,51.60%,"64 detected by practice, out of new diagnoses."
126285,47006,J81012,The Parkstone Tower Practice,Cancer Detection Rate,53.40%,"58 detected by practice, out of new diagnoses."
154868,49066,C88095,Mill Road Surgery,Cancer Detection Rate,42.90%,"35 detected by practice, out of new diagnoses."
46705,12495,F81732,Swanwood Partnership,Cancer Detection Rate,45.50%,"22 detected by practice, out of new diagnoses."


In [30]:
indicators[indicators['MetricName']=='Care Quality Commission Inspection Ratings'].sample(20)

,OrganisationID,OrganisationCode,OrganisationName,MetricName,Value,Text
194129,51876,Y00996,The Meridian Practice,Care Quality Commission Inspection Ratings,NaN,Good
26523,5408,C88008,Pitsmoor Surgery,Care Quality Commission Inspection Ratings,NaN,Good
132300,47465,G82048,Horsmans Place Partnership,Care Quality Commission Inspection Ratings,NaN,Good
227835,54318,H83028,Addington Medical Practice,Care Quality Commission Inspection Ratings,NaN,No rating
181466,50953,H81039,Park House Surgery,Care Quality Commission Inspection Ratings,NaN,No rating
163116,49660,M88627,Dr Bhadauria's,Care Quality Commission Inspection Ratings,NaN,Good
78532,15314,K82022,Kingswood Surgery,Care Quality Commission Inspection Ratings,NaN,Good
33423,11468,E81052,Kirby Road Surgery,Care Quality Commission Inspection Ratings,NaN,Good
9166,3812,B81635,Laurbel Surgery,Care Quality Commission Inspection Ratings,NaN,Good
60330,13662,G83044,Fairfield Pms,Care Quality Commission Inspection Ratings,NaN,Good
